In [3]:
import tensorflow as tf
import pandas as pd
import json
import numpy as np
import csv
import os
import matplotlib
import matplotlib.pyplot as plt
import pickle
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.externals import joblib
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from time import localtime
from datetime import datetime as dt
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist
from pathlib import Path


In [39]:
df = pd.read_csv('NN_Data/preproDataFull.csv')

In [52]:
LOGDIR = './graphs'
sess    = tf.Session()

# HYPER PARAMS
LEARN   = 0.01
BATCH   = 950
EPOCHS  = 10

# HIDDEN LAYERS
HL_1    = 10
HL_2    = 2


# SPLIT TRAIN AND TEST
Train, Test = train_test_split(df.copy(),test_size=.1,random_state=22)

# OTHER PARAMS
INPUT_S = Train.shape[1] - 1
N_CLASS = 950

In [53]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [62]:
with tf.name_scope('input'):
    games  = tf.placeholder(tf.float32, [None,INPUT_S], name='games')
    labels = tf.placeholder(tf.float32, [None,N_CLASS], name='labels')
def fc_layer(x,layer,size_out,activation=None):
    with tf.name_scope(layer):
        size_in   = int(x.shape[1])
        W         = tf.Variable(tf.random_normal([size_in,size_out]), name='weights')
        b         = tf.Variable(tf.constant(-1, dtype=tf.float32, shape=[size_out]), name='biases')
        
        wx_plus_b = tf.add(tf.matmul(x, W), b)
        if activation:
            return activation(wx_plus_b)
        return wx_plus_b
# layer inits
fc_1 = fc_layer(games, 'fc_1', HL_1, tf.nn.relu)
fc_2 = fc_layer(fc_1, 'fc_2', HL_2, tf.nn.relu)

# to prevent overfit
dropp = tf.nn.dropout(fc_2, keep_prob=0.9)
y     = fc_layer(dropp, 'output', N_CLASS)
with tf.name_scope('loss'): 
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y,labels=labels))
    tf.summary.scalar('loss',loss)
    
with tf.name_scope('optimizer'):   
    train = tf.train.AdamOptimizer(LEARN).minimize(loss)
    
    
with tf.name_scope('evaluation'):    
    correct  = tf.equal(tf.argmax(y,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))
    tf.summary.scalar('accuracy',accuracy)
train_writer = tf.summary.FileWriter(os.path.join(LOGDIR, "train"), sess.graph)
test_writer  = tf.summary.FileWriter(os.path.join(LOGDIR, "test"), sess.graph)


summary_op   = tf.summary.merge_all()

init = tf.global_variables_initializer()
sess.run(init)

with tf.name_scope('training'):    
    step = 0
    for epoch in range(EPOCHS):
        print('epoch',epoch,"\n=========\n")
        
        for batch in range(int(Train.shape[0]/BATCH)):
            step+=1
            
            Y_Train = np.array(list(Train['res']))
            Y_Test  = np.array(list(Test['res']))
            
            X_Train = Train.drop(['res'],axis=1).copy()
            X_Test  = Test.drop(['res'],axis=1).copy()
            
            X_Train = X_Train.to_numpy()
            X_Test  = X_Test.to_numpy()
            
            batch_xs, batch_ys = next_batch(BATCH,X_Train,Y_Train)
            
            summary_result, _ = sess.run( [summary_op, train], feed_dict={games: batch_xs, labels: batch_ys} )

            train_writer.add_summary(summary_result, step)
            summary_result, acc = sess.run( [summary_op, accuracy], feed_dict={games: X_Test, labels: Y_Test} )

            test_writer.add_summary(summary_result, step)

            print("Batch ", batch, ": accuracy = ", acc)

epoch 0 



ValueError: Cannot feed value of shape (950,) for Tensor 'input_29/labels:0', which has shape '(?, 950)'